In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import nltk

from nltk.corpus import stopwords
import math
import scipy.stats as stats
from statistics import *
import numpy as np

import os
import PyPDF2
import pandas as pd
import pickle

In [ ]:
#Set file paths

corp_path = 'C:/Project/corpus/'              #Path to corpus
temp_path = 'C:/Project/temp/'                #Path for temporary files created

In [ ]:
def extract_terms(vector,ngrams=(1,1)):
    vectorizer = TfidfVectorizer(ngram_range=ngrams,smooth_idf=True)
    tf = vectorizer.fit_transform(vector)
    idf = list(vectorizer.idf_)
    all_features_idf = dict(zip(vectorizer.get_feature_names(), idf))
    return all_features_idf


def terms_for_doc_table(file,text):
    vect_list = [line.lower() for line in text]
    # arguments vector name, ngram range
    singles = extract_terms(vect_list,(1,1))
    single_terms=[]
    stop = set(stopwords.words('english'))
    for key in singles:
        if key not in stop :
                if key.strip() not in ['',' ']:
                    single_terms.append(key)
    #print('File is', file)
    #print('Number of single terms selected from corpus : ',len(single_terms))

    
    # arguments vector name, ngram range
    corpus = extract_terms(vect_list,(2,4))
    print('Number of features extracted from corpus : ',len(corpus))

    #Retain only relevant terms which contain information about the data - remove very common words from the list.
    all_rel_terms=[]
    for key in corpus:
        #if min_idf+(2*sd_idf) < corpus[key]  :
                if key.strip() not in ['',' ']:
                    all_rel_terms.append(key)
    #print('Number of relevant features selected from corpus : ',len(all_rel_terms))
    
    return single_terms, all_rel_terms

In [ ]:
file_list = os.listdir(corp_path)

In [ ]:
#Create a document table dataframe
doc_table_list = []
id = 0


for file in file_list:
    #Extract text from pdf files
    # Known File read warning : PDF had issues during creation? 
    if '.pdf' in file :
        id = id + 1
        pdfFileObj = open(corp_path + file, 'rb')
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        #print(pdfReader.numPages)
        pageObj = pdfReader.getPage(0)
        text = pageObj.extractText()
        pdfFileObj.close()
        textlines = text.split("\n")
        singles, all_terms = terms_for_doc_table(file,textlines)
        doc_table_list.append([id,file, singles, all_terms])
       
    if '.txt' in file :
        id = id + 1
        textfile = open(corp_path + file, 'r')
        text = textfile.read()
        textlines = text.split("\n")
        textfile.close()
        singles, all_terms = terms_for_doc_table(file,textlines)
        doc_table_list.append([id,file, singles, all_terms])
        
       
      
    #more lines here for other file types

In [ ]:
doc_table = pd.DataFrame(doc_table_list, columns = ['ID', 'file_name', 'single_terms', 'phrases'])

In [ ]:
doc_table.to_pickle(temp_path + "CRA_doc_table.pkl")